In [21]:
import re
import subprocess

## Pylint messages

In [22]:
print(subprocess.check_output(['pylint', '--version']).decode())

pylint 1.7.2, 
astroid 1.5.3
Python 3.6.2 | packaged by conda-forge | (default, Jul 23 2017, 22:59:30) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]



In [23]:
output = subprocess.check_output(
    "pylint --list-msgs | grep '^:' | awk '{print $2}' | cut -c 2-6", 
    shell=True, universal_newlines=True)
pylint_messages = output.split()

In [24]:
len(pylint_messages)

221

## Ignored (.pylinrc) messages

In [25]:
import configparser
config = configparser.ConfigParser()
config.read('../pyta/python_ta/.pylintrc')
pylintrc_messages = [m.strip(",") for m in config['MESSAGES CONTROL']['disable'].split()]

In [26]:
len(pylintrc_messages)

117

## Undocumented messages

In [27]:
undocumented_messages = {
    'F0001',
    'I0001',
    'F0010',
    'F0002',
    'F0202',
}

## Covered messages

In [28]:
LINK_RE = re.compile('###.*\{#([A-Z][0-9]+)\}.*')  # looks for '{#E0601}'
HEADER_RE = re.compile('###.*\(([A-Z][0-9]+)\).*')  # looks for '### E0601'

In [29]:
link_messages = []
header_messages = []

with open('../website/index.md') as ifh:
    for line in ifh:
        link_messages += LINK_RE.findall(line)
        header_messages += HEADER_RE.findall(line)

In [30]:
assert link_messages == header_messages

In [31]:
covered_messages = link_messages

In [32]:
len(covered_messages)

115

## Missing messages

In [33]:
# Messages in index.md but not in pylint
set(covered_messages) - set(pylint_messages)

{'E9991', 'E9996', 'E9998', 'E9999'}

In [34]:
# Messages in index.md and in .pylintrc
set(covered_messages) & set(pylintrc_messages)

set()

In [35]:
# Messages produced by pylint that are not covered by .pylintrc or index.md
missing_messages = set(pylint_messages) - set(pylintrc_messages) - set(covered_messages) - undocumented_messages

In [36]:
len(missing_messages)

0

In [37]:
missing_message_text = {}

for code in missing_messages:
    print(code)
    message = subprocess.check_output("pylint --help-msg={}".format(code), universal_newlines=True, shell=True)
    missing_message_text[code] = message.strip() 

In [38]:
RE_MESSAGE_NAME = re.compile(":([a-zA-Z\-]+) \(([A-Z0-9]+)\):")

In [39]:
new_errors_text = """
## New errors {#new}

"""

for code, text in missing_message_text.items():
    match = RE_MESSAGE_NAME.findall(text)
    assert len(match) == 1
    name, code_ = match[0]
    assert code == code_
    text = """\
### {1} ({0}) {{#{0}}}

{3}

~~~~ {{include="{0}_{2}"}}
~~~~

""".format(code, name.replace('-', ' ').capitalize(), name.replace('-', '_'), text)
    new_errors_text += text

print(new_errors_text)


## New errors {#new}




In [40]:
RE_MESSAGE_NAME.findall(":superfluous-parens (C0325): *Unnecessary parens after %r keyword*")

[('superfluous-parens', 'C0325')]